# Распознавание эмоций

Классификатор - предобученная MobileNet на весах imagenet. В качестве детектора работает алгоритм Хаара 

In [76]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import cv2
import os
import glob
from datetime import datetime
from time import sleep
import numpy as np

In [78]:
# Пути

ROOT_PATH  = os.path.abspath('.')
TRAIN_DATA_PATH = os.path.join(ROOT_PATH, 'train')
VALIDATION_DATA_PATH = os.path.join(ROOT_PATH, 'validation')
STORED_FRAMES_DIR = 'data' # папка для сохранения snapshot'ов
OUTPUT_DIR = 'output'

# Сеттинги

IMG_SIZE = (224, 224, 3)
batch_size = 8
epochs = 50
class_labels = os.listdir(VALIDATION_DATA_PATH) # ['happy', 'sad'] # class names берем из структуры файлов


CLASS_DICT = {"1": "smile",
              "2": "sad",
              "3": "surprised",
              "4": "anger",
              "5": "boared",
              "6": "trick"
             }

num_classes = len(CLASS_DICT)

In [79]:
# Список изображений в датасете
def get_filenames_list(classes, path):
    dataset = []
    for cls in classes:
        print('Processing class: {}'.format(cls))
        for fpath in glob.glob(os.path.join(path, cls, '*.jpg')):
            cls_idx = classes.index(cls)
            dataset.append((fpath, cls_idx))
    print('Done')
    return dataset

In [80]:
n_train_samples = len(get_filenames_list(list(CLASS_DICT.values()), TRAIN_DATA_PATH))
n_val_samples = len(get_filenames_list(list(CLASS_DICT.values()), VALIDATION_DATA_PATH))

Processing class: smile
Processing class: sad
Processing class: surprised
Processing class: anger
Processing class: boared
Processing class: trick
Done
Processing class: smile
Processing class: sad
Processing class: surprised
Processing class: anger
Processing class: boared
Processing class: trick
Done


## Модуль обучения

#### Сбор и подготовка датасета 

In [81]:
def make_train_filestructure():
    
    # Создает директории с именами классов

    for cls in CLASS_DICT.values():
        d = os.path.join(ROOT_PATH, STORED_FRAMES_DIR, cls)
        if not os.path.exists(d):
            os.mkdir(d)
            print(f"Created {cls} dir")

make_train_filestructure()

Created smile dir
Created sad dir
Created surprised dir
Created anger dir
Created boared dir
Created trick dir


In [82]:
# Запись семплов с камеры

def run_cam_in_dataset_creation_mode():
    
    """
    Usage:
    Выбираем класс из controls, корчим соответствующую рожицу, сохраняем по 's'
    
    Controls: 
    q - exit
    s - take snapshot
    1 - choose "smile" class
    2 - choose "sad" class
    3 - choose "surprized" class
    4 - choose "anger" class
    5 - choose "boared" class
    6 - choose "trick" class
    """
    
    cap = cv2.VideoCapture(0)
    cur_class = CLASS_DICT.get('1')
    img_counter = 0
    while True:
        # Покадровое чтение захвата
        success, frame = cap.read()
        cv2.imshow('Emotion Detector',frame)
        
        # Options
        
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'): # wait for ESC key to exit
            break
        elif k == ord('1'):
            print(f"Class changed to {CLASS_DICT.get('1')}")
            cur_class = CLASS_DICT.get('1')
        elif k == ord('2'):
            print(f"Class changed to {CLASS_DICT.get('2')}")
            cur_class = CLASS_DICT.get('2')
        elif k == ord('3'):
            print(f"Class changed to {CLASS_DICT.get('3')}")
            cur_class = CLASS_DICT.get('3')
        elif k == ord('4'):
            print(f"Class changed to {CLASS_DICT.get('4')}")
            cur_class = CLASS_DICT.get('4')        
        elif k == ord('5'):
            print(f"Class changed to {CLASS_DICT.get('5')}")
            cur_class = CLASS_DICT.get('5')            
        elif k == ord('6'):
            print(f"Class changed to {CLASS_DICT.get('6')}")
            cur_class = CLASS_DICT.get('6')            
        elif k == ord('s'):
            fn_gen = f"frame{str(datetime.now()).replace(':','-')}{cur_class}.jpg"
            cv2.imwrite(f"./data/{cur_class}/{fn_gen}", frame)
            img_counter += 1
            print(f"Saved img # {img_counter} to {cur_class}")

    cap.release()
    cv2.destroyAllWindows()
    return

In [84]:
run_cam_in_dataset_creation_mode()

Class changed to anger
Saved img # 1 to anger
Saved img # 2 to anger
Saved img # 3 to anger
Saved img # 4 to anger
Saved img # 5 to anger
Saved img # 6 to anger
Saved img # 7 to anger
Saved img # 8 to anger
Saved img # 9 to anger


## Подготовка и обучение модели

In [85]:
# MobileNet работает с изображениями размерности 224, 224
MobileNet = tf.keras.applications.MobileNet(weights='imagenet',include_top=False,input_shape=IMG_SIZE)

In [86]:
# Set layers trainable
for layer in MobileNet.layers:
    layer.trainable = True

# print layers
# for (i,layer) in enumerate(MobileNet.layers):
#     print(str(i),layer.__class__.__name__,layer.trainable)

In [87]:
def addTopModelMobileNet(bottom_model, num_classes):
    
    """ Добавление  финальных слоев на выход модели"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model)
    
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

In [88]:
model = Model(inputs = MobileNet.input, outputs = FC_Head)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [89]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
                        TRAIN_DATA_PATH,
                        target_size = IMG_SIZE[:2],
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            VALIDATION_DATA_PATH,
                            target_size=IMG_SIZE[:2],
                            batch_size=batch_size,
                            class_mode='categorical')

Found 148 images belonging to 6 classes.
Found 38 images belonging to 6 classes.


In [90]:
checkpoint = ModelCheckpoint(
                             'my_emotions_model.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )


history = model.fit(
            train_generator,
            steps_per_epoch=n_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=n_val_samples//batch_size)


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 18 steps, validate for 3 steps
Epoch 1/50
17/18 [===========================>..] - ETA: 1s - loss: 2.0048 - accuracy: 0.3485
Epoch 00001: val_loss improved from inf to 2.19646, saving model to my_emotions_model.h5
18/18 [==============================] - 28s 2s/step - loss: 1.9729 - accuracy: 0.3643 - val_loss: 2.1965 - val_accuracy: 0.3750
Epoch 2/50
17/18 [===========================>..] - ETA: 0s - loss: 1.2953 - accuracy: 0.4167
Epoch 00002: val_loss did not improve from 2.19646
18/18 [==============================] - 18s 992ms/step - loss: 1.2513 - accuracy: 0.4429 - val_loss: 3.0324 - val_accuracy: 0.4583
Epoch 3/50
17/18 [===========================>..] - ETA: 0s - loss: 0.9316 - accuracy: 0.6742
Epoch 00003: val_loss improved from 2.19646 to 1.81232, saving model to my_emotions_model.h5
18/18 [==============================] - 17s 959ms/step - loss: 0.8862 - accuracy: 0.6929 - val_loss: 1.8123 - val_accuracy: 0.5000
E

Epoch 28/50
17/18 [===========================>..] - ETA: 0s - loss: 0.0131 - accuracy: 1.0000
Epoch 00028: val_loss improved from 0.10473 to 0.08867, saving model to my_emotions_model.h5
18/18 [==============================] - 17s 964ms/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.0887 - val_accuracy: 0.9583
Epoch 29/50
17/18 [===========================>..] - ETA: 0s - loss: 0.0146 - accuracy: 0.9924
Epoch 00029: val_loss did not improve from 0.08867
18/18 [==============================] - 18s 1s/step - loss: 0.0139 - accuracy: 0.9929 - val_loss: 0.1247 - val_accuracy: 0.9583
Epoch 30/50
17/18 [===========================>..] - ETA: 1s - loss: 0.0098 - accuracy: 0.9924
Epoch 00030: val_loss did not improve from 0.08867
18/18 [==============================] - 19s 1s/step - loss: 0.0094 - accuracy: 0.9929 - val_loss: 0.1298 - val_accuracy: 0.9583
Epoch 31/50
17/18 [===========================>..] - ETA: 1s - loss: 0.0192 - accuracy: 0.9926
Epoch 00031: val_loss did not impro

## Модуль Inference

cv2.CascadeClassifier parms:


**image** – исходное изображение

**scaleFactor** – определяет то, на сколько будет увеличиваться скользящее окно поиска на каждой итерации. 1.1 означает на 10%, 1.05 на 5% и т.д. Чем больше это значение, тем быстрее работает алгоритм.

**minNeighbors** — Чем больше это значение, тем более параноидальным будет поиск и тем чаще он будет пропускать реальные лица, считая, что это ложное срабатывание. Оптимальное значение 3-6.

**minSize** – минимальный размер лица на фото. 30 на 30 обычно вполне достаточно.

In [93]:
# Классификатора Хаара
face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
# Загрузка весов
classifiers = ['my_emotions_model.h5','emotion_face_mobilNet.h5', 'Emotion_Detection.h5']
classifier = load_model(classifiers[0])
class_labels = list(CLASS_DICT.values())

In [94]:
classifier.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [98]:
def run_cam_in_detection_mode():
    
    """Runs model and predicts label
    Stores snapshots by 's' key """
    
    cap = cv2.VideoCapture(0)

    while True:
        # Покадровое чтение захвата
        success, frame = cap.read()
        labels = []
        faces = face_classifier.detectMultiScale(frame,1.3,5)

        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = frame[y:y+h,x:x+w]
            roi_gray = cv2.resize(roi_gray,(224,224),interpolation=cv2.INTER_AREA)
            
            if np.sum([roi_gray])!=0:
                roi = roi_gray.astype('float')/255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi,axis=0)
                
                # make a prediction on the ROI, then lookup the class
                
                preds = classifier.predict(roi)[0]
                label=class_labels[preds.argmax()]
                label_position = (x,y)
                cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
                
            else:
                cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        cv2.imshow('Emotion Detector',frame)
        
        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'): # wait for ESC key to exit
            break
        elif k == ord('s'):
            fn_gen = f"frame{str(datetime.now()).replace(':','-')}.jpg"
            print(fn_gen)
            cv2.imwrite(f"./{OUTPUT_DIR}/{fn_gen}", frame)
            
            print('Image saved')

    cap.release()
    cv2.destroyAllWindows()
    return


In [100]:
run_cam_in_detection_mode()

frame2020-06-23 00-45-56.114093.jpg
Image saved
frame2020-06-23 00-46-01.614151.jpg
Image saved
frame2020-06-23 00-46-54.181024.jpg
Image saved
frame2020-06-23 00-46-58.461389.jpg
Image saved
frame2020-06-23 00-47-00.007531.jpg
Image saved
